In [1]:
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [2]:
 !pip install catboost holidays scikit-learn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.8 MB/s eta 0:00:00


In [5]:
!pip install --upgrade scikit-learn

In [4]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 12.5 MB/s eta 0:00:00


In [6]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# 데이터 불러오기
train_df = pd.read_csv("train.csv")

# 날짜 컬럼 처리 및 파생 변수 생성
train_df['일시'] = pd.to_datetime(train_df['일시'].astype(str), format="%Y%m%d")
train_df['year'] = train_df['일시'].dt.year
train_df['month'] = train_df['일시'].dt.month
train_df['day'] = train_df['일시'].dt.day
train_df['weekday'] = train_df['일시'].dt.weekday  # 0: 월요일, 6: 일요일
train_df['is_weekend'] = (train_df['weekday'] >= 5).astype(int)

# 컬럼명 영문화
train_df = train_df.rename(columns={
    '광진구': 'Gwangjin',
    '동대문구': 'Dongdaemun',
    '성동구': 'Seongdong',
    '중랑구': 'Jungnang'
})

In [7]:
# 공휴일 정보 파생 변수
import holidays
kr_holidays = holidays.KR(years=[2018, 2019, 2020, 2021, 2022])

# 임시공휴일 및 선거일 직접 추가
extra_holidays = {
    '2018-06-13': '제7회 전국동시지방선거',
    '2020-04-15': '제21대 국회의원선거',
    '2020-08-17': '임시공휴일',
    '2022-03-09': '제20대 대통령선거',
    '2022-06-01': '제8회 전국동시지방선거'
}
for date, name in extra_holidays.items():
    kr_holidays[date] = name


train_df['holiday_name'] = train_df['일시'].dt.date.map(kr_holidays)
train_df['holiday_name'] = train_df['holiday_name'].fillna('None')
train_df['is_holiday'] = (train_df['holiday_name'] != 'None').astype(int)

# 활동형/휴식형 공휴일 구분
activity_holidays = ['어린이날']
rest_holidays = ['설날', '추석', '크리스마스']
train_df['is_activity_holiday'] = train_df['holiday_name'].isin(activity_holidays).astype(int)
train_df['is_rest_holiday'] = train_df['holiday_name'].isin(rest_holidays).astype(int)

In [17]:
# Feature/Target/범주형 변수 정의
feature_cols = [
    'year', 'month', 'day', 'weekday', 'is_weekend',
    'is_holiday', 'holiday_name', 'is_activity_holiday', 'is_rest_holiday',
    ]
cat_features = [
    'month', 'weekday', 'is_weekend', 'is_holiday', 'holiday_name',
    'is_activity_holiday', 'is_rest_holiday',
       ]
target_cols = ['Gwangjin', 'Dongdaemun', 'Seongdong', 'Jungnang']


In [18]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import pandas as pd

results = {}
summary_data = []

# 학습/검증 인덱스 정의
train_idx = train_df['year'] < 2021    # 2018~2020년
val_idx = train_df['year'] == 2021     # 2021년

X_train = train_df.loc[train_idx, feature_cols]
X_val = train_df.loc[val_idx, feature_cols]

In [19]:
import optuna
import catboost as cb
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

# 예시: target_cols와 해당 구별로 train/val 데이터, cat_features는 미리 준비
# target_cols = ['Gwangjin', 'Dongdaemun', 'Seongdong', 'Jungnang']

# ‘최종 best params’ 사전
best_params_dict = {
'Gwangjin': {'learning_rate': 0.07463933708677424, 'depth': 5, 'l2_leaf_reg': 1.074777222256696,
'subsample': 0.805469615390896, 'colsample_bylevel': 0.7879853811582616},
'Dongdaemun': {'learning_rate': 0.05090532965560029, 'depth': 4, 'l2_leaf_reg': 2.4336961951246585,
'subsample': 0.9860732586553317, 'colsample_bylevel': 0.8329465092538842},
'Seongdong': {'learning_rate': 0.06865667868137201, 'depth': 5, 'l2_leaf_reg': 3.548575891319757,
'subsample': 0.7118705479838677, 'colsample_bylevel': 0.7334892361378472},
'Jungnang': {'learning_rate': 0.05691151851732044, 'depth': 4, 'l2_leaf_reg': 2.5595777632181074,
'subsample': 0.9751001756331396, 'colsample_bylevel': 0.5490555159536538}
}

results_summary = []
all_valid_preds = {}

for target in target_cols:
    params = {
        "iterations": 1000,
        "learning_rate": best_params_dict[target]["learning_rate"],
        "depth": best_params_dict[target]["depth"],
        "l2_leaf_reg": best_params_dict[target]["l2_leaf_reg"],
        "subsample": best_params_dict[target]["subsample"],
        "colsample_bylevel": best_params_dict[target]["colsample_bylevel"],
        "random_seed": 42,
        "cat_features": cat_features,
        "loss_function": "RMSE",
        "verbose": 0
    }
    y_train = train_df.loc[train_idx, target]
    y_val = train_df.loc[val_idx, target]
    model = cb.CatBoostRegressor(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)
    preds = model.predict(X_val)

    rmse = np.sqrt(mean_squared_error(y_val, preds))
    results_summary.append({
        '구': target,
        'Best RMSE': round(rmse, 4),
        'Best Params': best_params_dict[target]
    })

    all_valid_preds[target] = preds

# 7. 구별 성능 요약
summary_df = pd.DataFrame(results_summary)
print("\n=== 모든 구별 CatBoost Best RMSE & Params 요약 ===")
print(summary_df.to_markdown(index=False))




=== 모든 구별 CatBoost Best RMSE & Params 요약 ===
| 구         |   Best RMSE | Best Params                                                                                                                                                     |
|:-----------|------------:|:----------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Gwangjin   |      2.9355 | {'learning_rate': 0.07463933708677424, 'depth': 5, 'l2_leaf_reg': 1.074777222256696, 'subsample': 0.805469615390896, 'colsample_bylevel': 0.7879853811582616}   |
| Dongdaemun |      2.7365 | {'learning_rate': 0.05090532965560029, 'depth': 4, 'l2_leaf_reg': 2.4336961951246585, 'subsample': 0.9860732586553317, 'colsample_bylevel': 0.8329465092538842} |
| Seongdong  |      2.5493 | {'learning_rate': 0.06865667868137201, 'depth': 5, 'l2_leaf_reg': 3.548575891319757, 'subsample': 0.7118705479838677, 'colsample_bylevel': 0.7334892361378472}  |


In [20]:
valid_preds_df = pd.DataFrame(all_valid_preds, index=X_val.index)  # index는 날짜 등일 가능성 있음


valid_preds_df = valid_preds_df.reset_index()
valid_preds_df = valid_preds_df.rename(columns={'index': '일시'})

valid_preds_df.to_csv('catboost_valid_preds_2021.csv', index=False)
print("\ncatboost_valid_preds_2021.csv 파일로 저장 완료!")



catboost_valid_preds_2021.csv 파일로 저장 완료!
